test calling the model

In [15]:
from groq import Client
from os import getenv

client = Client(
    api_key = getenv('GROQ_API_KEY')
)
user_prompt = 'how to ask a woman out for a date?'

response = client.chat.completions.create(
    # mandatory parameters
    messages = [
        {
            'role' : 'system',
            'content' : 'you are a helpful assistant'
        },
        {
            'role' : 'user',
            'content' : user_prompt
        }
    ],
    model = 'llama-3.3-70b-versatile',
    # optional para meters
    # lower values of temperature are related to less randomness
    temperature = 0.8,
    # maximun number of tokens to generate
    max_completion_tokens = 600,
    # A stop sequence is a predefined or user-specified text string that
    # signals an AI to stop generating content, ensuring its responses
    # remain focused and concise. Examples include punctuation marks and
    # markers like "[end]".
    stop = None,
    # If set, partial message deltas will be sent.
    stream = False
    
)
print(response.choices[0].message.content, end = "")
# for streaming = True
# for chunk in response:
#     print(chunk.choices[0].delta.content, end = "")

Asking a woman out on a date can be nerve-wracking, but with some tips and advice, you can increase your chances of getting a positive response. Here's a step-by-step guide on how to ask a woman out for a date:

**Before you ask:**

1. **Get to know her**: Try to have a conversation with her and get to know her interests, hobbies, and values. This will help you find common ground and make the ask more personal.
2. **Make sure you're interested**: Ensure that you're genuinely interested in getting to know her better and that it's not just a casual ask.
3. **Be respectful**: Remember that she's a person with her own thoughts, feelings, and boundaries. Be respectful of her time and decision.

**The ask:**

1. **Choose the right time and place**: Find a private and comfortable setting where you both feel at ease. Avoid asking her out in front of a large group or in a noisy environment.
2. **Be confident and genuine**: Take a deep breath, be yourself, and speak clearly. Avoid using cheesy p

what how to add memory to the chat

In [20]:
chat_history = [
    {
        'role' : 'system',
        'content' : 'you are a helpful AI assistant'
    }
]
def chat_with_memory(
    user_prompt : str
):
    # adding the user input to the conversation history
    chat_history.append(
        {
            'role' : 'user',
            'content' : user_prompt
        }
    )
    response = client.chat.completions.create(
        messages = chat_history,
        model='llama-3.3-70b-versatile',
        temperature=0.8,
        max_completion_tokens=600,
        stop=None,
        stream=False
    )
    ai_response = response.choices[0].message.content
    # addding ai response to the conversation history
    chat_history.append(
        {
            'role' : 'assistant',
            'content' : ai_response
        }
    )
    return ai_response

questions = [
    'how to bake brownies',
    'what adjustments should I make to create a healthier version?',
]
for q in questions:
    print(f"user question:\n{q}")
    print('-'*60)
    print(chat_with_memory(q))
    print('-'*60)

user question:
how to bake brownies
------------------------------------------------------------
Baking brownies can be a delightful experience. Here's a simple recipe to get you started:

**Classic Fudgy Brownies Recipe**

Ingredients:

* 1 and 1/2 sticks of unsalted butter (12 tablespoons), plus more for greasing the pan
* 2 cups of sugar
* 4 large eggs
* 1/2 cup of unsweetened cocoa powder
* 1 teaspoon of vanilla extract
* 1/4 teaspoon of salt
* 1 and 1/4 cups of all-purpose flour
* 1 cup of semi-sweet chocolate chips
* Optional: nuts, espresso powder, or other mix-ins of your choice

Instructions:

1. **Preheat your oven**: Set your oven to 350°F (180°C). Make sure to adjust the oven rack to the middle position to ensure even baking.
2. **Prepare the pan**: Grease an 8-inch square baking pan with butter and line it with parchment paper. Leave some overhang on the sides for easy removal.
3. **Melt the butter and sugar**: In a medium saucepan, melt the butter and sugar over low heat,

adding a legal disclaimer

In [24]:
import json
# setting the system prompt
SYSTEM_PROMPT = """You are a helpful AI assistant. 
Your task is to provide a response to the user's question and classify whether the topic falls into:
- Legal
- Financial
- General (if it's neither legal nor financial)

Return a JSON object with:
- 'response': Your answer to the user.
- 'category': One of ['legal', 'financial', 'general'].
"""
# setting the disclaimer message
DISCLAIMER_MESSAGE = (
    "\n\n**Disclaimer:** Be mindful that this is an AI assistant. "
    "Please consult with a professional before proceeding."
)
# initialize the chat history
messages = [
        {
            'role' : 'system',
            'content' : SYSTEM_PROMPT
        }
    ]
def chat_with_memory(
    user_prompt : str
):
    # add prompt to the chat history
    messages.append(
        {
            'role' : 'user',
            'content' : user_prompt
        }
    )
    response = client.chat.completions.create(
        messages=messages,
        model="llama-3.3-70b-versatile",
        temperature=0.8,
        max_completion_tokens=600,
        stop=None,
        stream=False
    )
    ai_reply = response.choices[0].message.content
    try:
        json_format_response = json.load(ai_reply)
        answer = json_format_response.get('response', 'It was not possible to process the prompt')
        category = json_format_response.get('category', 'general').lower()
    except json.JSONDecodeError:
        answer = ai_reply
        category = 'general'
    
    if category in ['legal', 'financial']:
        answer += DISCLAIMER_MESSAGE
    # update the chat history
    messages.append(
        {
            'role' : 'assistant',
            'content' : answer
        }
    )
    return answer, category
    
user_prompt = "Can I sue my employer for wrongful termination?"
response, category = chat_with_memory(user_prompt)
print(f"Category: {category}\nResponse:\n{response}")

AttributeError: 'str' object has no attribute 'read'